In [ ]:
# Import necessary libraries
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolov11.pt")

# Open the video file
video_path = "samples/vietnam.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create VideoWriter object
video_name = video_path.split("/")[-1]
output_path = f'run/{video_name.split(".")[0]}_tracked.mp4'
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLO model tracking on the frame
        results = model.track(frame, persist=True, show=False)

        # Get the boxes and track IDs (with error handling)
        try:
            boxes = results[0].boxes
            track_ids = boxes.id.int().cpu().tolist() if boxes.id is not None else []
        except AttributeError:
            track_ids = []  # Handle case where tracking fails

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks only if we have valid tracking data
        if track_ids:
            for box, track_id in zip(boxes.xyxy.cpu().tolist(), track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 120:  # retain 30 tracks for the last 30 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.vstack(track).astype(np.int32).reshape(-1, 1, 2)
                cv2.polylines(
                    annotated_frame,
                    [points],
                    isClosed=False,
                    color=(230, 230, 230),
                    thickness=4,
                )

        # Write the frame to output video
        out.write(annotated_frame)
    else:
        # Break the loop if the end of the video is reached
        break

# Release everything
cap.release()
out.release()
print(f"Video has been saved to '{output_path}'")